### Imports & Data loading

In [29]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import plotly.io as pio

In [30]:
train = pd.read_csv('data/flights_train.csv')
X_test = pd.read_csv('data/flights_train.csv')
X_train = train.drop(["target"], axis=1)
y_train = train.target

### Exploratory data analysis 

In [31]:
num_features = ['avg_weeks', 'std_weeks']
date_features = ['flight_date']
cat_features = ['from', 'to']
features = ['target'] + date_features + cat_features + num_features


In [32]:
print(f'Dimenstion of our data (training data only) : {train.shape}')

Dimenstion of our data (training data only) : (8896, 6)


In [33]:
# count of missing values
train.isna().sum()

flight_date    0
from           0
to             0
avg_weeks      0
target         0
std_weeks      0
dtype: int64

In [34]:
# date feature inspection
print(
    f'Checking the boundaries of flight dates of testing : {X_test.flight_date.min(), X_test.flight_date.max()}')
print(
    f'Checking the boundaries of flight dates of training : {X_train.flight_date.min(), X_train.flight_date.max()}')


Checking the boundaries of flight dates of testing : ('2011-09-01', '2012-11-14')
Checking the boundaries of flight dates of training : ('2011-09-01', '2012-11-14')


#### Data visualisation

In [35]:
rows, cols = 3, 2
fig = make_subplots(
    rows=rows, cols=cols,
    subplot_titles=(features))

for i in range(rows):
    for j in range(cols):
        try:
            fig.add_trace(go.Histogram(x=train[features[2*i+j]]),
                          row=i+1, col=j+1)

        except:
            pass
fig.update_layout(title_text="Distribution of each variable in our data",
                  height=900)

fig.show()


In [36]:
rows, cols = 2, 2
fig = make_subplots(
    rows=rows, cols=cols,
    subplot_titles=(cat_features + date_features))

for i in range(rows):
    for j in range(cols):
        try:
            fig.add_trace(go.Histogram(x=train[(cat_features + date_features)[2*i+j]]),
                          row=i+1, col=j+1)
        except:
            pass
fig.update_layout(title_text="Sum of target variation depending on each categorical or date feature",
                  height=600)

fig.show()


In [37]:
figtest = px.scatter(train, y='target', x='avg_weeks')
figtest.show()

In [38]:
rows, cols = 2, 1
fig = make_subplots(
    rows=rows, cols=cols,
    subplot_titles=(num_features))

for i in range(rows):
    for j in range(cols):
        fig.add_trace(go.Scatter(x=train[num_features[i+j]], y=train['target'], mode='markers'),
                      row=i+1, col=j+1)
fig.update_layout(title_text="Target variation depending on each numerical feature",
                  height=900)

fig.show()


### Preprocessing 

In [39]:
# declare types
gro_dtypes = {
    'from': 'category',
    'to': 'category',
}
# import training data and testing data
training_data = pd.read_csv('data/flights_train.csv', dtype=gro_dtypes)
X_test = pd.read_csv('data/flights_Xtest.csv', dtype=gro_dtypes)


# Create variables month and day
training_data["year"] = pd.DatetimeIndex(training_data['flight_date']).year
training_data["month"] = pd.DatetimeIndex(training_data['flight_date']).month
training_data["day"] = pd.DatetimeIndex(training_data['flight_date']).day
X_test["year"] = pd.DatetimeIndex(X_test['flight_date']).year
X_test["month"] = pd.DatetimeIndex(X_test['flight_date']).month
X_test["day"] = pd.DatetimeIndex(X_test['flight_date']).day

# split data to X_train , X_test , y_train
X_train = training_data.drop(["target"], axis=1)
y_train = training_data.target
X_train = X_train.drop(["flight_date"], axis=1)
X_test = X_test.drop(["flight_date"], axis=1)

# X_train.set_index("flight_date",inplace=True)
# X_test.set_index("flight_date",inplace=True)
# the variable X contains the whole independent variables data (training plus testing)
X = pd.concat([X_test.assign(ind="test"), X_train.assign(ind="train")])

X_train_hot = pd.get_dummies(X_train, drop_first=True)
X_test_hot = pd.get_dummies(X_test, drop_first=True)


### comparing basic models


Random forest


In [40]:
# Random forest score with cross_validation
# oob_score: The default is False. Set this to True if you want to perform Out-of-bag (oob) evaluation which is an alternative to cross-validation.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
pipe = Pipeline([('scaler', StandardScaler()),
                ('regr', RandomForestRegressor(n_jobs=-1))])
n_scores = -cross_val_score(pipe, X_train_hot, np.ravel(
    y_train), scoring='neg_root_mean_squared_error', cv=4, n_jobs=-1, error_score='raise')
# report performance
print('RMSE: %.3f (+/- %.3f)' % (np.mean(n_scores), np.std(n_scores)))


RMSE: 0.734 (+/- 0.013)


XGBoost

In [41]:
# xgboost
from xgboost import XGBRegressor

# evaluate the model
pipe = Pipeline([('scaler', StandardScaler()), ('xgb', XGBRegressor(
    objective='reg:squarederror', eval_metric="rmse", subsample=0.5))])
n_scores = -cross_val_score(pipe, X_train_hot,  np.ravel(
    y_train), scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1, error_score='raise')
# report performance
print('RMSE: %.3f (+/- %.3f)' % (np.mean(n_scores), np.std(n_scores)))


RMSE: 0.712 (+/- 0.020)


AdaBoost

In [42]:
# adaboost
from sklearn.ensemble import AdaBoostRegressor

# evaluate the model with cross_val_score
pipe = Pipeline([('scaler', StandardScaler()), ('ada', AdaBoostRegressor())])
n_scores = -cross_val_score(pipe, X_train_hot, np.ravel(
    y_train), scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1, error_score='raise')
# report performance
print('RMSE: %.3f (%.3f)' %
      (np.mean(n_scores), np.std(n_scores)))


RMSE: 0.919 (0.012)


CatBoost

In [43]:
from catboost import CatBoostRegressor

cbr = CatBoostRegressor(
    loss_function='MAE',
    eval_metric='RMSE',
    cat_features=cat_features)
# evaluate the model with cross_val_score
n_scores = -cross_val_score(cbr, X_train, np.ravel(
    y_train), scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1, error_score='raise')
# report performance
print('RMSE: %.3f (%.3f)' %
      (np.mean(n_scores), np.std(n_scores)))

RMSE: 0.680 (0.014)


### Optimisation of best model

In [44]:
from skopt.space.space import Real, Integer
from skopt import BayesSearchCV
search_spaces = {
    'iterations': Integer(10, 1000),
    'depth': Integer(4, 10),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    # randomness for scoring splits
    'random_strength': Real(1e-9, 10, 'log-uniform'),
    # settings of the Bayesian bootstrap
    'bagging_temperature': Real(0.0, 1.0),
    # L2 regularization
    'l2_leaf_reg': Integer(2, 100),  
} 


In [45]:
# Wrapping everything up into the Bayesian optimizer
opt = BayesSearchCV(estimator=cbr,
                    search_spaces=search_spaces,
                    scoring="neg_root_mean_squared_error",
                    cv=5,
                    # max number of trials
                    n_iter=50,                                        
                    # number of hyperparameter sets evaluated at the same time
                    n_points=1,
                    # number of jobs
                    n_jobs=-1,                                        
                    # if not iid it optimizes on the cv score
                    iid=False,
                    return_train_score=False,
                    refit=False,
                    # optmizer parameters: we use Gaussian Process (GP)
                    optimizer_kwargs={'base_estimator': 'GP'},
                    # random state for replicability
                    random_state=0)                                   


C:\Users\DataScience\AppData\Local\Programs\Python\Python37\lib\site-packages\skopt\searchcv.py:300: UserWarning:

The `iid` parameter has been deprecated and will be ignored.



In [ ]:
# Running the optimizer
from skopt.callbacks import DeadlineStopper, DeltaYStopper
# We stop if the gain of the optimization becomes too small
overdone_control = DeltaYStopper(delta=0.005)
# We impose a time limit (6 hours)
time_limit_control = DeadlineStopper(total_time=60 * 60 * 7)
opt.fit(X_train, np.ravel(y_train), callback=[
        overdone_control, time_limit_control])

opt.best_params_

CatBoost took 2068.29 seconds,  candidates checked: 51, best CV score: -0.665 ± 0.029
Best parameters:
OrderedDict([('bagging_temperature', 0.004460588146218009), ('depth', 7), ('iterations', 1000), ('l2_leaf_reg', 77), ('learning_rate', 0.061881750957331255), ('random_strength', 6.633642794012173e-08)])


In [46]:
cbr_opti = CatBoostRegressor(
    loss_function = 'MAE',
    eval_metric = 'RMSE',
    cat_features = cat_features,
    iterations = 1000,
    learning_rate = 0.061881750957331255,
    depth = 7,
    l2_leaf_reg = 77,
    bagging_temperature = 0.004460588146218009,
    random_strength = 6.633642794012173e-08)

In [47]:
# evaluate the model with cross_val_score
n_scores = -cross_val_score(cbr_opti, X_train, np.ravel(
    y_train), scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1, error_score='raise')
# report performance
print('RMSE: %.3f (%.3f)' %
      (np.mean(n_scores), np.std(n_scores)))

RMSE: 0.663 (0.012)
